In [1]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

import numpy as np

from unidecode import unidecode as un

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
data = pd.read_csv('original_wec_data.csv')
data = data.drop('Unnamed: 0', axis =1)

#renaming several columns for clarity
data = data.rename(columns={"kph":"lap_avg_kph", "top_speed":"lap_max_kph", "number":"car_number"})

/tmp/ipykernel_1770/3822536386.py:1: DtypeWarning: Columns (26,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('original_wec_data.csv')


In [3]:
def make_season_year(df):
    
    new_column_values = []
        
    for i, row in data.iterrows():
        if row['season'] == 2012:
            new_column_values.append(2012)
        elif row['season'] == 2013:
            new_column_values.append(2013)
        elif row['season'] == 2014:
            new_column_values.append(2014)
        elif row['season'] == 2015:
            new_column_values.append(2015)
        elif row['season'] == 2016:
            new_column_values.append(2016)
        elif row['season'] == 2017:
            new_column_values.append(2017)
        elif row['season'] == '2017':
            new_column_values.append(2017)
        elif row['season'] == '2018-2019':
            if row['round'] <= 5:
                new_column_values.append(2018)
            elif row['round'] > 5:
                new_column_values.append(2019)
        elif row['season'] == '2019-2020':
            if row['round'] <= 4:
                new_column_values.append(2019)
            elif row['round'] > 4:
                new_column_values.append(2020)
        elif row['season'] == '2021':
            new_column_values.append(2021)
        elif row['season'] == 2021:
            new_column_values.append(2021)
        elif row['season'] == 2022:
            new_column_values.append(2022)
    
            
    df.insert(loc=26, column='year', value=new_column_values)
    
    return df

In [4]:
data = make_season_year(data)
# This function makes a new column for the year of each entry. The original season column contains several unuseable values. Two seaons ran through multiple years


In [5]:
def fix_driver_names(df):
    
    from unidecode import unidecode as un
    
   
    for i in range(len(df['driver_name'])):
        try:
            df['driver_name'][i] = un(df['driver_name'][i])
            df['driver_name'][i] = df['driver_name'][i].lower()
            df['driver_name'][i] = df['driver_name'][i].strip()
        except:
            print(i)
        
    return df

In [6]:
data = fix_driver_names(data)
# Cleans up driver names to allow for better matching when merging driver_ratings df

/tmp/ipykernel_1770/355984185.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['driver_name'][i] = un(df['driver_name'][i])
/tmp/ipykernel_1770/355984185.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['driver_name'][i] = df['driver_name'][i].lower()
/tmp/ipykernel_1770/355984185.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['driver_name'][i] = df['driver_name'][i].strip()


In [8]:
ratings_data = pd.read_csv('wec_entry_lists/all_driver_ratings.csv')
ratings_data = ratings_data.drop('Unnamed: 0', axis =1)

data = pd.merge(data, ratings_data, on=("driver_name", "year"), how="left")
data = data.set_index(data.index).reindex(data.index)


# Importing and Merging df from 'combining_ratings' notebook to add a column containing each drivers metal rating

In [9]:
def make_avg(df):
    df['rolling_5l_avg'] = df.groupby(['round', 'car_number', 'driver_number', 'driver_stint_no'])['lap_time_s'].rolling(window=5, min_periods=1).mean().reset_index(level=[0,1,2,3], drop=True)
    return df

In [10]:
data = make_avg(data)
# Creating a new column containing a rolling 5 lap average for each driver stint.


In [12]:
def column_cleaning(df):
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].fillna(0)
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].replace('B', 1)
#   B represents a car crossing the start/finish line in pitlane, NaN means the car was on track. Values are replaced with binary 0 for on-track, 1 for in-pit

    df['circuit'] = df['circuit'].replace({'SPA FRANCORCHAMPS':'Spa', 'BAHRAIN INTERNATIONAL CIRCUIT':'Bahrain', 'FUJI SPEEDWAY':'Fuji', 'SHANGHAI INTERNATIONAL CIRCUIT':'Shanghai', 'CIRCUIT OF THE AMERICAS':'COTA',
    'AUTODROMO HERMANOS RODRIGUEZ':'Mexico', 'LE MANS 2018':'Le Mans', 'SPA FRANCORCHAMPS 2019':'Spa','LE MANS 2019':'Le Mans', 'BAHRAIN INTERNATIONAL CIRCUIT 2019': 'Bahrain',
    'BAHRAIN INTERNATIONAL CIRCUIT 2020':'Bahrain', 'AUTODROMO DO ALGARVE':'Portimao', 'AUTODROMO NAZIONALE DI MONZA':'Monza', 'BAHRAIN INTERNATIONAL CIRCUIT 6 HOURS':'Bahrain', 'BAHRAIN INTERNATIONAL CIRCUIT 8 HOURS':'Bahrain'})
    df['circuit'] = df['circuit'].str.lower() 
#     Replacing the long-form and/or year appended circuit names with the common-use short versions. Also changes all to lower-case


    df = df.drop('flag_at_fl', axis=1)
#     df in column only exists for season 2022, some value meanings can not be determined.

    return df

In [13]:
data = column_cleaning(data)

In [14]:
data.head()

,car_number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,s3,s3_improvement,lap_avg_kph,elapsed,hour,s1_large,s2_large,s3_large,lap_max_kph,driver_name,pit_time,class,group,team,manufacturer,season,year,circuit,round,vehicle,team_no,lap_time_ms,lap_time_s,engine,driver_stint_no,driver_stint,team_stint_no,team_stint,elapsed_ms,position,class_position,interval_ms,interval,gap,class_interval,class_gap,elapsed_s,rating,rolling_5l_avg
0,57,1,1,0 days 00:02:23.141000,0,0,52.533,0,45.335,0,45.273,0,151.4,0 days 00:02:23.141000,10:33,0 days 00:00:52.533000,0 days 00:00:45.335000,0 days 00:00:45.273000,247.6,niclas jonsson,NaN,LMGTE Am,NaN,Krohn Racing,Ferrari,2012,2012,sebring,1,Ferrari 458 Italia GT2,Krohn Racing #57,143141.0,143.141,Ferrari,1,Niclas Jönsson Stint #1,1,Krohn Racing #57 Stint #1,143141,20.0,1.0,349.0,0.349,26.922,0.000,0.000,143.141,NaN,143.141
1,88,2,1,0 days 00:02:24.016000,0,0,52.915,0,45.342,0,45.759,0,150.4,0 days 00:02:24.016000,10:33,0 days 00:00:52.915000,0 days 00:00:45.342000,0 days 00:00:45.759000,220.4,paolo ruberti,NaN,LMGTE Am,NaN,Team Felbermayr-Proton,Porsche,2012,2012,sebring,1,Porsche 997 GT3-RSR,Team Felbermayr-Proton #88,144016.0,144.016,Porsche,1,Paolo Ruberti Stint #1,1,Team Felbermayr-Proton #88 Stint #1,144016,21.0,2.0,875.0,0.875,27.797,0.875,0.875,144.016,NaN,144.016
2,50,2,1,0 days 00:02:25.278000,0,0,53.568,0,45.577,0,46.133,0,149.1,0 days 00:02:25.278000,10:33,0 days 00:00:53.568000,0 days 00:00:45.577000,0 days 00:00:46.133000,233.6,pedro lamy,NaN,LMGTE Am,NaN,Larbre Competition,Chevrolet,2012,2012,sebring,1,Chevrolet Corvette C6.R GT2,Larbre Competition #50,145278.0,145.278,Chevrolet,1,Pedro Lamy Stint #1,1,Larbre Competition #50 Stint #1,145278,22.0,3.0,1262.0,1.262,29.059,1.262,2.137,145.278,NaN,145.278
3,70,2,1,0 days 00:02:25.548000,0,0,53.938,0,45.614,0,45.996,0,148.8,0 days 00:02:25.548000,10:33,0 days 00:00:53.938000,0 days 00:00:45.614000,0 days 00:00:45.996000,218.3,pascal gibon,NaN,LMGTE Am,NaN,Larbre Competition,Chevrolet,2012,2012,sebring,1,Chevrolet Corvette C6.R GT2,Larbre Competition #70,145548.0,145.548,Chevrolet,1,Pascal Gibon Stint #1,1,Larbre Competition #70 Stint #1,145548,23.0,4.0,270.0,0.270,29.329,0.270,2.407,145.548,NaN,145.548
4,61,2,1,0 days 00:02:25.859000,0,0,54.482,0,45.816,0,45.561,0,148.5,0 days 00:02:25.859000,10:33,0 days 00:00:54.482000,0 days 00:00:45.816000,0 days 00:00:45.561000,224.0,rui aguas,NaN,LMGTE Am,NaN,AF Corse-Waltrip,Ferrari,2012,2012,sebring,1,Ferrari 458 Italia GT2,AF Corse-Waltrip #61,145859.0,145.859,Ferrari,1,Rui Aguas Stint #1,1,AF Corse-Waltrip #61 Stint #1,145859,24.0,5.0,311.0,0.311,29.640,0.311,2.718,145.859,NaN,145.859


In [15]:
data.to_csv('final_wec_data.csv')

#Exporting the feature_engineered data to csv. This file will be read into the main machine learning project notebook.